# SIFT for defect classification in Rayleigh-Bénard Convection

We implement a SIFT (scale-invariant feature transform)-like algorithm for classifying defects in simulated Rayleigh-Bénard convection patterns. Instead of using the image features (a.k.a. keypoints) from the original SIFT algorithm, we decorate the temperature fields with topological and persistent homology defects and use these as keypoints. These keypoints are mathematically justified for the domain of interest, and solve the problem of finding relevant features in the absence of high-contrast regions which form edges and other typical features used in object recognition. What follows is a summary of (http://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf), with comments on how we are adapting the ideas for this problem.

## The SIFT Algorithm

The SIFT algorithm essentially builds out vectors of local image features. These features are built as follows:

* **Scale-space extrema detection:** This step in the algorithm is the most expensive and serves to find the keypoints, or points of interest in the image. These points are rotation and scale-invariant. For our purposes, this expensive step is performed through finding the topological and persistent homology defects. Additionally, we only desire rotation invariance and not scale-invariance, as these indicate different patterns in the temperature field. Rather than assigning scale to each keypoint, we assign the defect type as a feature. Of course, each keypoint also contains its location.

* **Keypoint localization:** A winnowing out of spurious keypoints. In our case, this is accomplished two ways. For topological defects, we increase the amount of smoothing prior to computing the orientation field. For persistent homology defects, we adjust the regions on the persistence plane and perhaps remove redundant points by clustering at small scales.

* **Orientation assignment:** This appears to be HOGs applied locally to each keypoint region. Binning is in 10 degree increments, so that 360 degrees of orientation requires 36 bins. Additionally, the gradient magnitudes are also used in order to weight the histogram. We use the orientation field rather than the gradient field, and discard magnitudes. Plus, we work on 180 degrees, and so we would have 18 bins. The histogram of orientations is then "fit" for a prominent direction. Essentially, the most prominent peak in the orientation histogram is assigned to the keypoint, and then additional copies of the keypoint are assigned for each peak that is within 80% of this maximum peak. This way, multi-modal distributions of orientations generate multiple keypoints, which they claim lends stability in the matching. This makes sense--mean-centering at zero as I naively proposed had the problem of the orientation favoring only the maximum peak, which is unstable with even small perturbations of the temperature field in the event that the orientation histogram was bimodal.

* **Keypoint descriptor:** After locating keypoints and assigning orientations, additional localized descriptions are added. The general idea is to impose a 2D coordinate system local to the keypoint identified. This coordinate system also seeks to adjust for changes in lightness and 3D vantage point. First, the orientation of the keypoint is used to center the image geometrically, and then the coordinate system is constructed relative to this alignment. After alignment, the local image patch is subdivided into smaller regions, within which coarsely-binned weighted histograms of orientations are constructed. Thus, the size of the local feature vector is $rn^2$, where $r$ is the number of bins in the coarse orientation histogram, and $n$ is the width of the $n \times n$ array of local subdivisions.

Thus, in total, our feature vector at each keypoint will contain:
* location (x,y)
* type (e.g. topological defect type, persistent homology defect type)
* orientation
* keypoint descriptor vector ($rn^2$ dimensions)

## Pattern classification

Although the original implementation of SIFT was for purposes of object detection, we use it instead for unsupervised image classification. In either case, in this step of the algorithm we must match keypoints that are similar between two collections of datasets. Since we are matching at the local image patch level, our databases of keypoints are very small, containing anywhere from zero to just 10 keypoints. Thus, even in a high-dimensional space (for large $rn^2$), we are able to use exact matching for keypoints. The distance used is the regular Euclidean distance and we find the nearest neighbors.

Somehow, the Hough transform is used to match clusters of keypoints that all match up mutually in their relative positions with one another. This is called the pose of the object and I don't understand it yet. Something about least squares. Something about using the reference image patch as the fixed orientation and then voting for the correct pose with the matched patch.

## Our Implementation

* First we run the SIFT-like algorithm for the topological and persistent homology keypoints to generate feature vectors at each keypoint.
* We then compute local image patches that we desire to classify. These could be centered at the keypoints themselves or by some other method. We define a distance between local image patches using methods from the SIFT paper. That is, metrics that are typically used to confirm or reject a match will instead be used to give a distance between two local image patches. We then perform these distance computations pairwise for a large collection of local image patches. This distance matrix is fed through the diffusion map embedding or LLE to map the set of local image patches to Euclidean space.
* A Gaussian mixture model using mean-field variational inference is then used to cluster these datapoints into many clusters. This completes the building of the dictionary of defects.
* Reserved test data is then used to find the nearest neighbor matches to the training dataset of local image patches, and clusters are assigned based on the cluster type of the nearest neighbor.

* TO DO: Figure out what the plots mean in the Lowe paper for justifying all of the choices, especially Section 6.2, descriptor testing.
* TO DO: Figure out how to compute a distance between two local image patches using the ideas from the SIFT paper.
* TO DO: Make a visualization that shows what we're doing and also to verify that the code is working as expected.


## Remaining questions

* How does this translate to experimental data? 
* How does this relate to the Lyapunov vectors?